In [2]:
import pandas as pd
import numpy as np

In [56]:
GSS = pd.read_csv("GSS.csv")
pd.set_option('display.max_columns', None)
print(GSS.shape)
GSS.head(2)

# Selecting relevant variables
columns_to_keep = ['year', 'id_', 'ballot', # Identifiers
    'happy', 'satjob', 'life', 'health', 'satfin',  # Satisfaction-related columns
    'age', 'sex', 'race', 'marital', 'educ', 'region',  # Demographics
    'wrktype', 'wrksched', 'hrs2', 'weekswrk', 'indus10', 'occ10', 'occ10_code', 'MPROFF', 'MPROFF_Label', 'conrinc', 'wrkslf',  # Work info
    'jobpromo', 'learnnew', 'workfast', 'opdevel', 'fairearn']  # Job quality

GSS_filtered= GSS[columns_to_keep]
GSS_filtered.shape

(17749, 60)


(17749, 30)

Data Cleaning

In [57]:
# Replaces all non-response values with N/A
non_response_values = [".i:  Inapplicable", ".n:  No answer", ".d:  Do not Know/Cannot Choose", ".s:  Skipped on Web"]
GSS_filtered = GSS_filtered.replace(non_response_values, np.nan)

GSS_filtered[GSS_filtered['wrktype'].notna()] #This is our primary dependent variable, so we need responses to this question

missing_percent = GSS_filtered.isna().mean()
#print(missing_percent)
GSS_filtered = GSS_filtered.drop(['life', 'health'], axis=1)
# remove columns with excessive missing responses
GSS_filtered.shape


(17749, 28)